<a href="https://colab.research.google.com/github/AndreiSugeac/Criptare-Decriptare/blob/master/SVM_SurgicalMaskDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!unzip '/content/drive/My Drive/ML_Datasets/ml-fmi-23-2020.zip'

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import read
import librosa

In [0]:
import matplotlib.pyplot as plt
import librosa.display

def extract_features(file_name):
  # we're loading the .wav file using the librosa load function
  data, sr = librosa.load(file_name, sr=None, res_type='kaiser_best')
  # representing the data as mfcc
  mfcc_conv = librosa.feature.mfcc(data, sr, n_mfcc=30)
  # transforming our mfcc into an 1D np array so it's easier to interpret
  mfcc = np.mean(mfcc_conv.T, axis=0)
  return mfcc

In [0]:
import os
import pandas as pd

def get_data(f_name, l_name):
  features = []
  files = os.listdir(f_name)

  # extracting the lables for the data
  lables = pd.read_csv(l_name, sep=',', header=None)
  lables.columns = ['File Name', 'Lable']

  # iterating through the files and setting up the data while extracting the features
  for temp_file in files:
    file_name = os.path.join(f_name, temp_file)
    mfcc_feature = extract_features(file_name)
    #getting the coresponding lable for the file
    lable = lables.loc[lables['File Name'] == temp_file].iloc[0]['Lable']
    features.append([mfcc_feature, lable])

  return features

In [0]:
# here we get the training data
train = get_data('/content/train/train', '/content/train.txt')

In [0]:
# here we get the testing data
test = get_data('/content/validation/validation', '/content/validation.txt')

In [0]:
train_dataframe = pd.DataFrame(train, columns=['Features', 'Lable'])
print(train_dataframe)
test_dataframe = pd.DataFrame(test, columns=['Features', 'Lable'])

In [0]:
train_data = np.array(train_dataframe.Features.tolist())
print(train_data)
train_lables = np.array(train_dataframe.Lable.tolist())

test_data = np.array(test_dataframe.Features.tolist())
test_lables = np.array(test_dataframe.Lable.tolist())

In [0]:
# defining our model
from sklearn import svm

# we use C to define the optimal margin 
C = 100
gamma = 1e-4
model = svm.SVC(C, kernel="rbf", gamma=gamma, verbose=1)
model.fit(train_data, train_lables)
predict_lable_train = model.predict(train_data)
print(predict_lable_train)
print("Training accuracy: ",model.score(train_data, train_lables))
print("Testing accuracy: ", model.score(test_data, test_lables))

In [0]:
ent = os.listdir('/content/test/test')
data_to_eval = []
for val in ent:
  testing_name = os.path.join('/content/test/test', val)
  feature = extract_features(testing_name)
  data_to_eval.append(feature)

print(data_to_eval)

In [0]:
data_ev = np.array(data_to_eval)
lables_predicted = model.predict(data_ev)
print(lables_predicted)
print(lables_predicted[2998])

In [0]:
f = open("predictions.txt", "a")
f.write("name,label\n")
ind = 0
for name in ent:
  f.write(str(name) + ',' + str(lables_predicted[ind]) + "\n")
  ind += 1

f.close()

In [0]:
import csv

with open("csvPrediction.csv", 'x') as fl:
  writer = csv.writer(fl)
  writer.writerow(["name", "label"])
  ind = 0
  for name in ent:
    writer.writerow([str(name), str(lables_predicted[ind])])
    ind += 1